# Eventalign_collapse usage

This program collapses the raw file generated by `nanopolish eventalign` by kmers rather than by event.

## Options

* **input_fn**

Path to a `nanopolish eventalign` tsv output file (read access required). In command line mode it is also possible to pipe the output of the `nanopolish eventalign` directly into `Eventalign_collapse`.

* **outdir**

Path to an existing directory where to write all the output files generated by `Eventalign_collapse`(write access required). If the directory does not exist an error is raised.

* **outprefix**

Prefix for all the files generated by the program

* **write_samples**

Write the concatenated sample data values corresponding to each kmer in the output data file. This options make only sense if `nanopolish eventalign` was ran with `--samples` option.
  
* **max_reads**

Controls the maximum number of read to parse before stopping to read the input file. This could be usefull for testing or downsampling. 

* **stat_fields**

Specify the list of statistical analyses to perform on a kmer basis and add in the output data file. This will only be performed if `nanopolish eventalign` was ran with `--samples` option.

!!! note "Valid statistics fields"
    * mean (mean of signal intensity)
    * median (mean of signal intensity)
    * std (standard deviation of signal intensity))
    * mad (median absolute deviation of signal intensity))
    * num_signals (Number of raw signal data point)

* **threads**

`Eventalign_collapse` is multi threaded to speed up the data processing and keep pace with Nanopolish if using the direct piping strategy. Take advantage of many threads if you have access to a large compute cluster

## Output files format

Contrary to `nanopolish eventalign` output text file, in `Eventalign_collapse` the reads are separated by a hashtag headers containing the read_id and ref_id. This reduces the redundancy and makes it easier to find the start and end of a read.

Example : ```#7ef1d7b9-5824-4382-b23b-78d82c07ebbd	YHR055C.```

The main data file contains the following fields:

* **ref_pos**: Reference sequence ID (contig).
* **ref_kmer**: Sequence of the reference kmers.
* **num_events**: Number of events for this kmer before collapsing.
* **dwell_time**: dwell time for this kmer in seconds
* **NNNNN_dwell_time**: dwell time of events for this kmers with a model sequence "NNNNN" (events ignored by nanopolish HMM).
* **mismatch_dwell_time**:  dwell time of events for this kmers with a model sequence different from the reference kmer
* **start_idx**: Only if nanopolish eventalign called with --signal_idx. Start coordinate on original raw signal in fast5 file
* **end_idx**: Only if nanopolish eventalign called with --signal_idx. End coordinate on original raw signal in fast5 file
* **mean**: Only if nanopolish eventalign called with --samples. Mean of the normalised signal values provided by Nanopolish eventalign
* **median**: Only if nanopolish eventalign called with --samples. Median of the normalised signal values provided by Nanopolish eventalign
* **std**: Only if nanopolish eventalign called with --samples. Standard deviation of the normalised signal values provided by Nanopolish eventalign
* **mad**: Only if nanopolish eventalign called with --samples. Median absolute deviation of the normalised signal values provided by Nanopolish eventalign
* **num_signals**: Only if nanopolish eventalign called with --samples. Number of raw signal points.
* **samples**: Only if nanopolish eventalign called with --samples and Eventalign_collapse called with --write_samples. List of normalised signal intensity values for this kmer

In addition `Eventalign_collapse` also generates an useful index file containing reads level information. It contains the following fields:

* **read_id**: Name or index of the read
* **ref_id**: Name of the reference sequence the read was aligned on (contig)
* **ref_start**: Start coordinate of the alignment on the reference sequence
* **ref_end**: End coordinate of the alignment on the reference sequence
* **dwell_time**: Cumulative dwell time in seconds for the entire resquiggled sequence
* **kmers**: Overall number of resquiggled kmers
* **NNNNN_kmers**: Number of resquiggled kmers containing at least 1 event for which the model sequence was "NNNNN"
* **mismatching_kmers**: Number of resquiggled kmers containing at least 1 event for which the model sequence diverged from the reference sequence
* **missing_kmers**: Number of skipped/missing reference positions in nanopolish output
* **byte_offset**: Number of characters before the start of the sequence in the main output file. **This can be used in conjunction with file.seek() to directly access the start of a read**. An example is provided in the Usage notebook.
* **byte_len**: Length of characters after byte_offset to the end of the read, excluding the last newline. **This can be used in conjunction with read() to read all the text chunk corresponding to the read**.

## Bash command line usage

### Command line help

In [1]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon Nanopolish_0.11.1

NanopolishComp Eventalign_collapse --help

usage: NanopolishComp Eventalign_collapse [-h] [-i INPUT_FN] [-o OUTDIR]
                                          [-p OUTPREFIX] [-s] [-r MAX_READS]
                                          [-f STAT_FIELDS [STAT_FIELDS ...]]
                                          [-t THREADS] [-v | -q]

Collapse the nanopolish eventalign output at kmers level and compute kmer
level statistics

optional arguments:
  -h, --help            show this help message and exit
  -v, --verbose         Increase verbosity (default: False)
  -q, --quiet           Reduce verbosity (default: False)

Input/Output options:
  -i INPUT_FN, --input_fn INPUT_FN
                        Path to a nanopolish eventalign tsv output file. If
                        '0' read from std input (default: 0)
  -o OUTDIR, --outdir OUTDIR
                        Path to the output folder (default: ./)
  -p OUTPREFIX, --outprefix OUTPREFIX
                        text outprefix for all the files generated (default:
                  

### Example usage

#### From an existing nanopolish eventalign output to a file

In [2]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon Nanopolish_0.11.1

NanopolishComp Eventalign_collapse -i ./data/eventalign_collapse//nanopolish_reads.tsv -o ./output/eventalign_collapse/
head ./output/eventalign_collapse/out_eventalign_collapse.tsv
head ./output/eventalign_collapse/out_eventalign_collapse.tsv.idx

#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time
1656	GAAAA	1	0.00266	0.0	0.0
1657	AAAAC	1	0.00764	0.0	0.0
1658	AAACA	1	0.00398	0.0	0.0
1659	AACAA	1	0.00432	0.0	0.0
1660	ACAAA	1	0.00498	0.0	0.0
1661	CAAAG	1	0.00564	0.0	0.0
1662	AAAGA	1	0.00963	0.0	0.0
1663	AAGAT	1	0.00299	0.0	0.0
ref_id	ref_start	ref_end	read_id	kmers	dwell_time	NNNNN_kmers	mismatch_kmers	missing_kmers	byte_offset	byte_len
YGR240C	1656	2960	0	1250	13.788570000000009	35	0	54	0	38028
YCR030C	1578	2576	1	971	11.487010000000005	23	0	27	38029	29704
YHR174W	0	839	2	825	9.659210000000002	15	0	14	67734	24231
YHR174W	218	1309	3	1028	11.06325000000001	36	0	63	91966	30801
YHR174W	462	1309	4	818	10.73776000000001	18	0	29	122768	24589
YLR441C	173	764	5	554	5.556939999999999	20	0	37	147358	16437
YGR192C	1	989	6	927	11.731470000000003	37	0	61	163796	27388
YDR500C	9	252	8	231	2.9179100000000027	7	0	14	191185	6751
YGR192C	3	995	7	946	12.30464000000001	31	0	50	197937	28123


Checking arguments
Testing output dir writability
Starting to process files
21 reads [00:00, 118.91 reads/s]
[Eventalign_collapse] total reads: 21 [116.81 reads/s]



#### From standard input to a file

In [3]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon Nanopolish_0.11.1

cat ./data/eventalign_collapse//nanopolish_reads_index.tsv | NanopolishComp Eventalign_collapse -o ./output --verbose
head ./output/eventalign_collapse/out_eventalign_collapse.tsv
head ./output/eventalign_collapse/out_eventalign_collapse.tsv.idx

#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time
1656	GAAAA	1	0.00266	0.0	0.0
1657	AAAAC	1	0.00764	0.0	0.0
1658	AAACA	1	0.00398	0.0	0.0
1659	AACAA	1	0.00432	0.0	0.0
1660	ACAAA	1	0.00498	0.0	0.0
1661	CAAAG	1	0.00564	0.0	0.0
1662	AAAGA	1	0.00963	0.0	0.0
1663	AAGAT	1	0.00299	0.0	0.0
ref_id	ref_start	ref_end	read_id	kmers	dwell_time	NNNNN_kmers	mismatch_kmers	missing_kmers	byte_offset	byte_len
YGR240C	1656	2960	0	1250	13.788570000000009	35	0	54	0	38028
YCR030C	1578	2576	1	971	11.487010000000005	23	0	27	38029	29704
YHR174W	0	839	2	825	9.659210000000002	15	0	14	67734	24231
YHR174W	218	1309	3	1028	11.06325000000001	36	0	63	91966	30801
YHR174W	462	1309	4	818	10.73776000000001	18	0	29	122768	24589
YLR441C	173	764	5	554	5.556939999999999	20	0	37	147358	16437
YGR192C	1	989	6	927	11.731470000000003	37	0	61	163796	27388
YDR500C	9	252	8	231	2.9179100000000027	7	0	14	191185	6751
YGR192C	3	995	7	946	12.30464000000001	31	0	50	197937	28123


Options summary
	package_name: NanopolishComp
	package_version: 0.6.1
	timestamp: 2019-05-02 10:32:46.350455
	quiet: False
	verbose: True
	threads: 4
	stat_fields: ['mean', 'median', 'num_signals']
	write_samples: False
	max_reads: None
	outprefix: out
	outdir: ./output
	input_fn: 0

Checking arguments
	Testing input file readability
Testing output dir writability
	Checking number of threads
	Checking if stat_fields names are valid
Starting to process files
	[split_reads] Start reading input file/stream
	[process_read 1] Starting processing reads
	[process_read 2] Starting processing reads
	[write_output] Start rwriting output
12 reads [00:00, 119.55 reads/s]	[split_reads] Done
	[process_read 1] Done
	[process_read 2] Done
21 reads [00:00, 116.86 reads/s]
	[write_output] Done
[Eventalign_collapse] total reads: 21 [115.22 reads/s]



#### On the fly, from nanopolish eventalign to a file

In [4]:
%%bash

# Load local bashrc and activate virtual environment
source ~/.bashrc
workon Nanopolish_0.11.1

nanopolish eventalign -t 4 --samples --scale-events --print-read-name --reads ./data/eventalign_collapse//reads.fastq --bam ./data/eventalign_collapse//aligned_reads.bam --genome ./data/eventalign_collapse//reference.fa | NanopolishComp Eventalign_collapse -o ./output
head ./output/eventalign_collapse/out_eventalign_collapse.tsv
head ./output/eventalign_collapse/out_eventalign_collapse.tsv.idx

#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time
1656	GAAAA	1	0.00266	0.0	0.0
1657	AAAAC	1	0.00764	0.0	0.0
1658	AAACA	1	0.00398	0.0	0.0
1659	AACAA	1	0.00432	0.0	0.0
1660	ACAAA	1	0.00498	0.0	0.0
1661	CAAAG	1	0.00564	0.0	0.0
1662	AAAGA	1	0.00963	0.0	0.0
1663	AAGAT	1	0.00299	0.0	0.0
ref_id	ref_start	ref_end	read_id	kmers	dwell_time	NNNNN_kmers	mismatch_kmers	missing_kmers	byte_offset	byte_len
YGR240C	1656	2960	0	1250	13.788570000000009	35	0	54	0	38028
YCR030C	1578	2576	1	971	11.487010000000005	23	0	27	38029	29704
YHR174W	0	839	2	825	9.659210000000002	15	0	14	67734	24231
YHR174W	218	1309	3	1028	11.06325000000001	36	0	63	91966	30801
YHR174W	462	1309	4	818	10.73776000000001	18	0	29	122768	24589
YLR441C	173	764	5	554	5.556939999999999	20	0	37	147358	16437
YGR192C	1	989	6	927	11.731470000000003	37	0	61	163796	27388
YDR500C	9	252	8	231	2.9179100000000027	7	0	14	191185	6751
YGR192C	3	995	7	946	12.30464000000001	31	0	50	197937	28123


[warning] fast5 file is unreadable and will be skipped: ./data/fast5//20180625_FAH77625_MN23126_sequencing_run_S1_57529_read_10_ch_319_strand.fast5
HDF5-DIAG: Error detected in HDF5 (1.10.4) thread 140205845075712:
  #000: H5F.c line 509 in H5Fopen(): unable to open file
    major: File accessibilty
    minor: Unable to open file
  #001: H5Fint.c line 1400 in H5F__open(): unable to open file
    major: File accessibilty
    minor: Unable to open file
  #002: H5Fint.c line 1546 in H5F_open(): unable to open file: time = Thu May  2 10:32:47 2019
, name = './data/fast5//20180625_FAH77625_MN23126_sequencing_run_S1_57529_read_10_ch_243_strand.fast5', tent_flags = 0
    major: File accessibilty
    minor: Unable to open file
  #003: H5FD.c line 734 in H5FD_open(): open failed
    major: Virtual File Layer
    minor: Unable to initialize object
  #004: H5FDsec2.c line 346 in H5FD_sec2_open(): unable to open file: name = './data/fast5//20180625_FAH77625_MN23126_sequencing_run_S1_57529_read_10_

## Python API usage

### Import the package

In [5]:
# Import main program
from NanopolishComp.Eventalign_collapse import Eventalign_collapse

# Import helper functions
from NanopolishComp.common import jhelp, head

### python API help

In [6]:
jhelp(Eventalign_collapse)

---

**NanopolishComp.Eventalign_collapse.__init__**

Collapse the nanopolish eventalign output by kmers rather that by events. kmer level statistics (mean, median, std, mad) are only computed if nanopolish is run with --samples option

---

* **input_fn** *: str (required)*

Path to a nanopolish eventalign tsv output file.

* **outdir** *: str (default = ./)*

Path to the output folder

* **outprefix** *: str (default = out)*

text outprefix for all the files generated

* **max_reads** *: int (default = None)*

Maximum number of read to parse. 0 to deactivate (default = 0)

* **write_samples** *: bool (default = False)*

If given, will write the raw sample if nanopolish eventalign was ran with --samples option

* **stat_fields** *: list of str (default = ['mean', 'median', 'num_signals'])*

List of statistical fields to compute if nanopolish eventalign was ran with --samples option. Valid values = "mean", "std", "median", "mad", "num_signals"

* **threads** *: int (default = 4)*

Total number of threads. 1 thread is used for the reader and 1 for the writer (default = 4)

* **verbose** *: bool (default = False)*

Increase verbosity

* **quiet** *: bool (default = False)*

Reduce verbosity



### Example usage

#### Example with minimal file

In [7]:
input_fn = "./data/eventalign_collapse//nanopolish_reads.tsv"
outdir = "./output/eventalign_collapse"
outprefix = "basic"

Eventalign_collapse (input_fn=input_fn, outdir=outdir, outprefix=outprefix, threads=6)

head("./output/eventalign_collapse/basic_eventalign_collapse.tsv")
head("./output/eventalign_collapse/basic_eventalign_collapse.tsv.idx")

Checking arguments
Testing output dir writability
Starting to process files
21 reads [00:00, 83.37 reads/s]
[Eventalign_collapse] total reads: 21 [78.29 reads/s]



#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time
1656	GAAAA	1	0.00266	0.0	0.0
1657	AAAAC	1	0.00764	0.0	0.0
1658	AAACA	1	0.00398	0.0	0.0
1659	AACAA	1	0.00432	0.0	0.0
1660	ACAAA	1	0.00498	0.0	0.0
1661	CAAAG	1	0.00564	0.0	0.0
1662	AAAGA	1	0.00963	0.0	0.0
1663	AAGAT	1	0.00299	0.0	0.0

ref_id  ref_start ref_end read_id kmers dwell_time         NNNNN_kmers mismatch_kmers missing_kmers byte_offset byte_len 
YGR240C 1656      2960    0       1250  13.788570000000009 35          0              54            0           38028    
YCR030C 1578      2576    1       971   11.487010000000005 23          0              27            38029       29704    
YHR174W 0         839     2       825   9.659210000000002  15          0              14            67734       24231    
YHR174W 218       1309    3       1028  11.06325000000001  36          0              63            91966       30801    
YLR441C 173       764     5       554   5.556939999999999  20         

#### Example with indexes

In [8]:
input_fn = "./data/eventalign_collapse//nanopolish_reads_index.tsv"
outdir = "./output/eventalign_collapse"
outprefix = "index"

Eventalign_collapse (input_fn=input_fn, outdir=outdir, outprefix=outprefix, threads=6)

head("./output/eventalign_collapse/index_eventalign_collapse.tsv")
head("./output/eventalign_collapse/index_eventalign_collapse.tsv.idx")

Checking arguments
Testing output dir writability
Starting to process files
21 reads [00:00, 92.53 reads/s]
[Eventalign_collapse] total reads: 21 [87.47 reads/s]



#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time	start_idx	end_idx
1656	GAAAA	1	0.00266	0.0	0.0	63446	63454
1657	AAAAC	1	0.00764	0.0	0.0	63423	63446
1658	AAACA	1	0.00398	0.0	0.0	63411	63423
1659	AACAA	1	0.00432	0.0	0.0	63398	63411
1660	ACAAA	1	0.00498	0.0	0.0	63383	63398
1661	CAAAG	1	0.00564	0.0	0.0	63366	63383
1662	AAAGA	1	0.00963	0.0	0.0	63337	63366
1663	AAGAT	1	0.00299	0.0	0.0	63328	63337

ref_id  ref_start ref_end read_id kmers dwell_time         NNNNN_kmers mismatch_kmers missing_kmers byte_offset byte_len 
YGR240C 1656      2960    0       1250  13.788570000000009 35          0              54            0           53046    
YCR030C 1578      2576    1       971   11.487010000000005 23          0              27            53047       41374    
YHR174W 0         839     2       825   9.659210000000002  15          0              14            94422       33595    
YHR174W 218       1309    3       1028  11.06325000000001  36          0      

#### Example including samples

In [9]:
input_fn = "./data/eventalign_collapse//nanopolish_reads_index.tsv"
outdir = "./output/eventalign_collapse"
outprefix = "stats"

Eventalign_collapse (input_fn=input_fn, outdir=outdir, outprefix=outprefix, threads=6, quiet=True, stat_fields=["mean", "std", "median", "mad", "num_signals"])

head("./output/eventalign_collapse/stats_eventalign_collapse.tsv")
head("./output/eventalign_collapse/stats_eventalign_collapse.tsv.idx")

[Eventalign_collapse] total reads: 21 [106.77 reads/s]



#0	YGR240C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time	start_idx	end_idx
1656	GAAAA	1	0.00266	0.0	0.0	63446	63454
1657	AAAAC	1	0.00764	0.0	0.0	63423	63446
1658	AAACA	1	0.00398	0.0	0.0	63411	63423
1659	AACAA	1	0.00432	0.0	0.0	63398	63411
1660	ACAAA	1	0.00498	0.0	0.0	63383	63398
1661	CAAAG	1	0.00564	0.0	0.0	63366	63383
1662	AAAGA	1	0.00963	0.0	0.0	63337	63366
1663	AAGAT	1	0.00299	0.0	0.0	63328	63337

ref_id  ref_start ref_end read_id kmers dwell_time         NNNNN_kmers mismatch_kmers missing_kmers byte_offset byte_len 
YGR240C 1656      2960    0       1250  13.788570000000009 35          0              54            0           53046    
YCR030C 1578      2576    1       971   11.487010000000005 23          0              27            53047       41374    
YHR174W 0         839     2       825   9.659210000000002  15          0              14            94422       33595    
YHR174W 462       1309    4       818   10.73776000000001  18          0      

#### Example including samples and writing samples values in ouput file

In [10]:
input_fn = "./data/eventalign_collapse//nanopolish_reads_samples.tsv"
outdir = "./output/eventalign_collapse"
outprefix = "samples"

Eventalign_collapse (input_fn=input_fn, outdir=outdir, outprefix=outprefix, threads=6, quiet=True, stat_fields=["mean", "std", "median", "mad", "num_signals"], write_samples=True)

head("./output/eventalign_collapse/samples_eventalign_collapse.tsv")
head("./output/eventalign_collapse/samples_eventalign_collapse.tsv.idx")

[Eventalign_collapse] total reads: 21 [11.5 reads/s]



#1	YCR030C
ref_pos	ref_kmer	num_events	dwell_time	NNNNN_dwell_time	mismatch_dwell_time	mean	std	median	mad	num_signals	samples
1578	CCACC	1	0.00365	0.0	0.0	70.06341552734375	2.593737840652466	70.67729949951172	1.4680023193359375	11	70.6773,72.1454,71.4114,63.6306,69.2093,69.2093,67.3008,72.5858,69.9433,72.8794,71.705
1579	CACCC	15	0.09761000000000002	0.0	0.0	74.59468841552734	3.2702603340148926	75.08149719238281	2.4958038330078125	294	75.0815,75.8156,79.6326,78.0177,75.3752,77.2837,78.0177,72.2922,76.99,74.7879,75.3752,75.522,75.6688,73.3199,74.9347,77.5773,73.7603,74.3475,72.7326,75.522,77.2837,75.2284,76.6964,78.4581,78.4581,79.7794,76.5496,76.8432,70.2369,70.2369,81.1006,77.8709,74.9347,72.439,75.8156,77.5773,75.9624,77.4305,76.1092,77.2837,64.5114,74.4943,78.4581,77.4305,76.8432,75.0815,76.1092,78.3113,76.5496,75.6688,76.5496,77.4305,78.6049,76.4028,75.8156,77.1368,71.8518,76.4028,75.8156,75.9624,74.9347,78.3113,72.2922,75.6688,75.0815,75.522,75.9624,70.6773,69.9433,69.2093,68.3284

## Using the index to random access a specific entry in the file

In [11]:
output_fn = "./output/eventalign_collapse/stats_eventalign_collapse.tsv"
index_fn = "./output/eventalign_collapse/stats_eventalign_collapse.tsv.idx"

# Import the index in a pandas dataframe (because it is simple)
import pandas as pd 
index_df = pd.read_csv (index_fn, sep="\t")

# Select random lines
random_lines = index_df.sample(5)
print ("Random index lines")
display (random_lines)

# Open the collapsed event align file
with open (output_fn) as fp:
    for id, read in random_lines.iterrows():
        
        # Access the header corresponding to the randomly selected index line using seek 
        fp.seek(0) # Return to file start
        fp.seek(read.byte_offset) # Move to the offset indicated in the index file
        print (fp.readline().rstrip()) # Print read header
        df = pd.read_csv (fp, nrows=read.kmers, sep="\t") # Read lines corresponding to the read
        with pd.option_context("display.max_rows",4): # display first and last lines
            display(df)

Random index lines


,ref_id,ref_start,ref_end,read_id,kmers,dwell_time,NNNNN_kmers,mismatch_kmers,missing_kmers,byte_offset,byte_len
10,YIL117C,27,953,10,889,13.64007,23,0,37,331734,37225
13,YPL198W,10,719,13,667,8.79275,30,0,44,409081,27943
7,YDR500C,9,252,8,231,2.91791,7,0,14,267232,9541
6,YGR192C,1,989,6,927,11.73147,37,0,61,228702,38529
15,YLR110C,14,395,15,364,4.61444,9,0,17,450178,15023


#10	YIL117C


,ref_pos,ref_kmer,num_events,dwell_time,NNNNN_dwell_time,mismatch_dwell_time,start_idx,end_idx
0,27,GGCCT,4,0.03287,0.00498,0.0,57497,57596
1,31,TTCCA,1,0.00398,0.00000,0.0,57485,57497
...,...,...,...,...,...,...,...,...
887,951,GAGTA,3,0.01163,0.00432,0.0,16588,16623
888,952,AGTAA,5,0.02590,0.00000,0.0,16510,16588


#13	YPL198W


...

,ref_pos,ref_kmer,num_events,dwell_time,NNNNN_dwell_time,mismatch_dwell_time,start_idx,end_idx
0,9,GGTAC,2,0.01095,0.00000,0.0,22637,22670
1,10,GTACT,1,0.00631,0.00000,0.0,22618,22637
...,...,...,...,...,...,...,...,...
229,248,TAAGG,2,0.01560,0.01228,0.0,13899,13946
230,251,GGCTA,1,0.00598,0.00000,0.0,13881,13899


#6	YGR192C


,ref_pos,ref_kmer,num_events,dwell_time,NNNNN_dwell_time,mismatch_dwell_time,start_idx,end_idx
0,1,TGGTT,1,0.01461,0.00000,0.0,45267,45311
1,2,GGTTA,1,0.00631,0.00000,0.0,45248,45267
...,...,...,...,...,...,...,...,...
925,987,GCCAA,2,0.01992,0.00000,0.0,10052,10112
926,988,CCAAG,5,0.02589,0.01228,0.0,9974,10052


#15	YLR110C


,ref_pos,ref_kmer,num_events,dwell_time,NNNNN_dwell_time,mismatch_dwell_time,start_idx,end_idx
0,14,TGTCG,2,0.00598,0.0,0.0,23697,23715
1,15,GTCGC,1,0.00730,0.0,0.0,23675,23697
...,...,...,...,...,...,...,...,...
362,393,TTGTT,7,0.04217,0.0,0.0,9871,9998
363,394,TGTTG,5,0.01859,0.0,0.0,9815,9871
